LangGraph self-corrective code assistant
We wanted to test the general idea of iterative code generation in LangGraph, making a few simplifications relative to the AlphaCodium work:

We start with a set of documentation specified by a user
We use a long context LLM to ingest it, and answer a question based upon it
We perform two layers of checking: we check imports to see if hallucinations were introduced
We check code execution to determine if the code is able to be executed without error
Checking for valid imports and execution is a reasonable stating point for code testing on open-ended questions related to a codebase.

In [1]:
%%capture --no-stderr
%pip install -U langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph faiss-cpu

### Documentation
As a test case, let's load docs related to LangChain Expression Language (LCEL)

In [6]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/expression_language/" 
loader = RecursiveUrlLoader(
    url = url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# LCEL w/ PydanticOutputParser ( outside the primary LCEL docs)
url = "https://python.langchain.com/docs/modules/model_io/output_parsers/quick_start"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup( x, "html.parser").text
)
docs_pydantic = loader.load()

# LCEL w/ Self Query ( out side the primary LCEL docs)
url="https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/"
loader = RecursiveUrlLoader(
    url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text 
)
docs_self_query = loader.load()

## Add docs
docs.extend([*docs_pydantic, *docs_self_query])

# Sort the list base on the URLs in 'metedata' -> 'source' 
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))

# Concatednate the 'page_content' of each sorted dictionary
concatenate_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)



### State
Our state is a dict that will contain keys (errors, question, code generation) relevant to code generation.

In [7]:
from typing import Dict, TypedDict

class GraphState(TypedDict):
    """ 
    Represents the state of our graph.
    Attributes:
        keys: A dictionary where each key is a string.
    """
    key: Dict[str, any]

### Graph
Our graph lays out the logical flow shown in the figure above.

In [ ]:
from operator import itemgetter

from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI

def Generate(state: GraphState):
    """ 
    Generate a code solution base on LCEL docs and the question
    with optional feedback from code execution tests

    Args: 
        state (dict): The current graph state

    Returns:
        state (dict): New key add to state, documents, that contacts retrieved documents
    """

    ## State 
    state_dict = state["key"]
    question = state_dict["question"]
    iter = state_dict["interations"]

    ## Data model
    class code(BaseModel):
        """Code output"""

        prefix: str = Field(description="Description for the problem and approach")
        s





